In [ ]:
print("Spark session is Loaded !!!")
sdf = spark.read.csv('hdfs://ec2-18-223-23-172.us-east-2.compute.amazonaws.com:8020/user/root/temp.csv')
for i in range(0,2500):
    #sdf = sdf.union(sdf)
    #sdf.write.csv(path="large_temp_2.csv")
    sdf.write.save(path="large_temp_2.csv", format='csv', mode='append')

VBox()

In [ ]:
new_sdf = spark.read.csv("large_temp_2.csv",header=True,encoding='utf-8')
new_sdf.count()

##### New approach:

In [ ]:
def doubleIT():
    sdf = spark.read.csv("large_temp_2.csv")
    sdf.write.save(path="large_temp_2.csv", format='csv', mode='append')
    
for i in range(0,10):
    doubleIT()
    
    
sdf = spark.read.csv("large_temp_2.csv")
sdf.count()

* How to know the large_temp_2.csv FOLDER size?
    * **Ans**: hadoop fs -du -s -h /user/livy/large_temp_2.csv
		* 34.2 M  /user/livy/large_temp_2.csv
            * from the new approach of doubleIT, it should reach a TB atleast
                * I had to stop the doubleIT execution as it was approaching 75% hadoop capacity but i think system did rollback of some last right because of which it came back 57% hdfs usage.
                
    *  hdfs dfs -df -h
    ```
        Filesystem                                                  Size     Used  Available  Use%
        hdfs://ip-172-31-39-242.us-east-2.compute.internal:8020  497.8 G  182.5 G    297.6 G   37%
    ```

* can 2 jobs, one for append and one for count work in parallel ? 
    * when I tried 2 spark submit (from 2 cells) in HUE notebook, both spark job started together but job with "count" stopped till "append" job completed. I could not find any uncommitted read option.
    
    
* what is time to read and count for csv file?
    * It did not read the file completely even if the spark job ran for 15 min.
    * I requested for 100 instances but that instance request got queued up and did not complete. I am wondering why the instance request got queued up. May be the resource manager , yarn, was busy scheduling the heavy IO operation for csv read.
    * yarn, in its last few logs in the HUE notebook cell output indicated the number of executors
    ```
    YARN Diagnostics:
19/05/02 20:03:00 INFO ExecutorAllocationManager: Requesting 3217 new executors because tasks are backlogged (new desired total will be 7312)

    ```
        * Because of this huge demand, abrupt closure resulted in huge rollback.
        * **What is the relation between instances and executors?** I think initial spark job log talks about it. 
        ```
        19/05/03 04:26:13 INFO RMProxy: Connecting to ResourceManager at ip-172-31-39-242.us-east-2.compute.internal/172.31.39.242:8032
        19/05/03 04:26:13 INFO Client: Requesting a new application from cluster with 46 NodeManagers
        19/05/03 04:26:13 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (11520 MB per container)
        19/05/03 04:26:13 INFO Client: Will allocate AM container, with 896 MB memory including 384 MB overhead
        19/05/03 04:26:13 INFO Client: Setting up container launch context for our AM
        19/05/03 04:26:13 INFO Client: Setting up the launch environment for our AM container
        19/05/03 04:26:13 INFO Client: Preparing resources for our AM container
        .
        .
        19/05/03 04:27:15 INFO DAGScheduler: Got job 0 (csv at NativeMethodAccessorImpl.java:0) with 7312 output partitions
        .
        .
        19/05/03 04:27:16 INFO DAGScheduler: Submitting 7312 missing tasks from ResultStage 0 (MapPartitionsRDD[2] at csv at NativeMethodAccessorImpl.java:0) (first 15 tasks are for partitions Vector(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14))
19/05/03 04:27:16 INFO YarnScheduler: Adding task set 0.0 with 7312 tasks
        .
        .
        .
        19/05/03 04:27:34 INFO TaskSetManager: Finished task 0.0 in stage 0.0 (TID 0) in 1948 ms on ip-172-31-40-6.us-east-2.compute.internal (executor 175) (1/7312)

        ```
        
        
        * Here 44 nodes are from the "spot" booked task nodes and 2 core nodes 
            * core nodes are presistent nodes and are created when EMR is iniialized. The also seem to define the storage capacity for the HDFS because 
                * if I try stop it, AWS throws warning indicating loss of data stored. 
                * and also "hdfs dfs -df -h" seems to sum up the storage capacity of core nodes.
                
        * Here, number of tasks are directly proportional to ouput partitions and it is difficult to map instances with tasks becuase completion of the previous task and the task itself may vary within the instance.


* How long did write to ORC file took?
    * 26 minutes and file size got reduced to 1.1 GB. Note that CSV file was around 300GB.
    * It took only 26 seconds to read the file and get the count ...wow !!! 
    * Now the question is ...Are we doing apple to apple comparison?
        * i.e. would csv file of 1.1 gb size can read and count in 26 seconds. It is understood that record count will be different but does the read capacity change? My intuition says yes csv file can read in the same speed as task executor of similar capacity. Raw fetch time should remains the same. 
            * Indeed my experiments wih 1.1GB csv data read and count showd fantastic result : 3 secs of execution time. But records count where drastically down.


* what is time to read and count for parquete file?
    * I have to know how to rename the columns in spark dataframe???
        ```
        par_sdf = spark.read.orc("large_temp_2_avro")
        old_columns = par_sdf.columns
        new_columns = []
        for c in old_columns:
            new_columns.append(c.lower().replace(" ","_").replace("'",""))

        print(new_columns)

        par_new_df = reduce(lambda data, idx: data.withColumnRenamed(old_columns[idx], new_columns[idx]) , xrange(len(old_columns)), par_sdf)
        par_new_df.show()
        par_new_df.write.parquet("large_temp_parquet")
        ```
        * Above piece of code converted the buggy colulmn name saved in ORC file and saves the dataframe (from ORC read) to parquet file format.
        * Parquet file Write operation took total of 20 minutes. File size is same as that of ORC = 1.1 GB
        * Read and count took mere 14 seconds
    
    
    
* can panda hold the csv or parquet file ?
    * if yes, can it operate on a "query" with the same speed as that of spark dataframe?
        * obviously not, otherwise spark should have closed its shutter !!! but let us see the difference and hail spark
    * if no, can we slice the dataset using spark df and may be panda simply does the plot( then why not matplot lib?)

